In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from netCDF4 import Dataset
import geopandas as gpd
import shapely
from scipy.ndimage import gaussian_filter

In [2]:
masks = {}

for path in os.listdir('../data/basin-outlines/CE/'):
    glacier = path.split('/')[-1].replace('.geojson', '')
    masks[glacier] = gpd.read_file('../data/basin-outlines/CE/' + path)
for path in os.listdir('../data/basin-outlines/CW/'):
    glacier = path.split('/')[-1].replace('.geojson', '')
    masks[glacier] = gpd.read_file('../data/basin-outlines/CW/' + path)
for path in os.listdir('../data/basin-outlines/SW/'):
    glacier = path.split('/')[-1].replace('.geojson', '')
    masks[glacier] = gpd.read_file('../data/basin-outlines/SW/' + path)

masks.keys()

dict_keys(['rolige-brae', 'eielson-gletsjer', 'graah-gletscher', 'bredegletsjer', 'kista-dan-gletsjer', 'sydbrae', 'magga-dan-gletsjer', 'dode-brae', 'charcot-gletscher', 'daugaard-jensen-gletsjer', 'vestfjord-gletsjer', 'kangilernata-sermia', 'sermeq-kullajeq', 'eqip-sermia', 'sermeq-avannarleq', 'kangiata-nunaata-sermia', 'akullersuup-sermia', 'narsap-sermia'])

In [3]:
files = []

for path in os.listdir('../data/igm-inputs'):
    files.append('../data/igm-inputs/' + path)

files

['../data/igm-inputs/graah-gletscher.nc',
 '../data/igm-inputs/daugaard-jensen-gletsjer.nc',
 '../data/igm-inputs/magga-dan-gletsjer.nc',
 '../data/igm-inputs/sermeq-avannarleq.nc',
 '../data/igm-inputs/charcot-gletscher.nc',
 '../data/igm-inputs/eqip-sermia.nc',
 '../data/igm-inputs/vestfjord-gletsjer.nc',
 '../data/igm-inputs/narsap-sermia.nc',
 '../data/igm-inputs/eielson-gletsjer.nc',
 '../data/igm-inputs/sermeq-kullajeq.nc',
 '../data/igm-inputs/kista-dan-gletsjer.nc',
 '../data/igm-inputs/dode-brae.nc',
 '../data/igm-inputs/bredegletsjer.nc',
 '../data/igm-inputs/akullersuup-sermia.nc',
 '../data/igm-inputs/kangiata-nunaata-sermia.nc',
 '../data/igm-inputs/kangilernata-sermia.nc',
 '../data/igm-inputs/rolige-brae.nc',
 '../data/igm-inputs/sydbrae.nc']

In [8]:
for f in files:
    ds = Dataset(f)

    glacier = f.split('/')[-1].replace('.nc', '')
    series = masks[glacier]

    xs, ys = np.meshgrid(ds.variables['x'], ds.variables['y'])
    xs += series.geometry.centroid[0].x
    ys += series.geometry.centroid[0].y

    mask = np.zeros_like(xs)

    for i in np.arange(xs.shape[0]):
        for j in np.arange(xs.shape[1]):
            point = shapely.Point(xs[i, j], ys[i, j])
            if series.geometry.contains(point)[0]:
                mask[i, j] = 1
            else:
                mask[i, j] = 0
    
    plt.imshow(mask)
    plt.show()

    break


    
    Us = np.sqrt(ds.variables['uvelsurfobs'][:]**2 + ds.variables['vvelsurfobs'][:]**2)

    S = ds.variables['usurfobs'][:]
    for i in range(10):
        S = gaussian_filter(S, sigma = 10)

    grad_vector = np.gradient(S, np.diff(ds.variables['x'])[0])
    gradS = np.sqrt(grad_vector[0]**2 + grad_vector[1]**2)

    gamma = 2.2e-5
    Ud = gamma * ds.variables['thkobs'][:]**4 * np.abs(gradS)**3

    fig, ax = plt.subplots(figsize = (12, 6))
    im = ax.imshow(np.where(ds.variables['thkobs'][:] > 0.5, np.where(Us > Ud, Us - Ud, 0.0), np.nan))
    plt.colorbar(im)
    plt.title(glacier.replace('-', ' ').capitalize())
    plt.show()

0
1
2
3
4
5
6
7


KeyboardInterrupt: 